#**Welcome to our Severity Index:**
This tutorial will go over what ECMWF model data we collected and how we constructed our Severity Index to construct which areas were affected the most during a major storm.
